In [2]:
import cv2
import sys
import numpy as np
import glob

## configuration

In [ ]:
file_name = './goolgenet_fig/space_shuttle.jpg'

img = cv2.imread(file_name)

if img is None:
    print('image read failed')
    sys.exit()
    
### Caffe model import 
model = 'bvlc_googlenet.caffemodel'
config = 'deploy.prototxt'

net = cv2.dnn.readNet(model,config)
# print(type(net))

classNames = []
with open('classification_classes_ILSVRC2012.txt', 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
    
# print(classNames)

### Deep learning
# blobFromImage(image[, scalefactor[, size(224, 224) defalt[, mean(104, 117, 123) defalt[, swapRB(False면 BGR)[, crop[, ddepth]]]]]]) -> retval
blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123),
                            swapRB = False)

net.setInput(blob)
prob = net.forward()
# print(prob[0, :10])

### 디멘션 평탄화
out = prob.flatten()
# print(out.shape)

### max valuse 찾기
classId = np.argmax(out)
print(classId)
confidence = out[classId]
print(confidence)
class_text = classNames[classId]
print(class_text)

text = f'{class_text} ({confidence*100:.2f}%)'
# print(text)
cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2,
           cv2.LINE_AA)




cv2.imshow('image', img)
cv2.waitKey()
cv2.destroyAllWindows()

## 과제

In [ ]:
file_name = glob.glob('./my_images/*.*')

# cv2.namedWindow('scene', cv2.WINDOW_NORMAL)
# cv2.setWindowProperty('scene', cv2.WND_PROP_FULLSCREEN,
#                      cv2.WINDOW_FULLSCREEN)


idx = 0

while True:
    img = cv2.imread(file_name[idx])
    
    
    model = 'bvlc_googlenet.caffemodel'
    config = 'deploy.prototxt'

    net = cv2.dnn.readNet(model,config)

    classNames = []
    with open('classification_classes_ILSVRC2012.txt', 'rt') as f:
        classNames = f.read().rstrip('\n').split('\n')

    blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123),
                                    swapRB = False)

    net.setInput(blob)
    prob = net.forward()

    out = prob.flatten()

    classId = np.argmax(out)
    print(classId)
    confidence = out[classId]
    print(confidence)
    class_text = classNames[classId]
    print(class_text)

    text = f'{class_text} ({confidence*100:.2f}%)'
    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2,
                cv2.LINE_AA)



    
    if img is None:
        print('image read failed')
        break
    cv2.imshow('scene', img)
    
    
    if cv2.waitKey(4000) == 27:
        break
        
    idx +=1
    if idx >= len(file_name):
        idx = 0

    




# cv2.imshow('image', img)
# cv2.waitKey()
cv2.destroyAllWindows()

## face detector

In [ ]:
img = cv2.imread('../face_detector/PS15122400030.jpg')

model = '../face_detector/opencv_face_detector_uint8.pb'
config = '../face_detector/opencv_face_detector.pbtxt.txt'
face_net = cv2.dnn.readNet(model, config)

blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123), swapRB = False)
face_net.setInput(blob)
out = face_net.forward()

detect = out[0, 0, :, :]
# print(detect.shape)

### image shape을 반환 받음
h, w = img.shape[:2]

for i in range(detect.shape[0]):
    confidence = detect[i, 2]
    
    if confidence > 0.5:
        
        x1 = int(detect[i, 3]*w)
        y1 = int(detect[i, 4]*h)
        x2 = int(detect[i, 5]*w)
        y2 = int(detect[i, 6]*h)
        
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
        text = 'Face: {}%'.format(round(confidence*100, 2))
        cv2.putText(img, text, (x1, y1-1), cv2.FONT_HERSHEY_COMPLEX,
                   1, (0, 0, 255), 1, cv2.LINE_AA)

cv2.imshow('image', img)
# print(type(out))
# print(out.shape)

cv2.waitKey()
cv2.destroyAllWindows()

## 과제

In [3]:
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print('camera open failed')
    sys.exit()
    
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS)*0.8)
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    
out = cv2.VideoWriter('output.avi', fourcc,fps, (w,h)) 

while True:
    
    ret, frame = cap.read()
    
    model = '../face_detector/opencv_face_detector_uint8.pb'
    config = '../face_detector/opencv_face_detector.pbtxt.txt'
    face_net = cv2.dnn.readNet(model, config)

    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123), swapRB = False)
    face_net.setInput(blob)
    out = face_net.forward()

    detect = out[0, 0, :, :]
    # print(detect.shape)

    ### image shape을 반환 받음
#     h, w = img.shape[:2]

    for i in range(detect.shape[0]):
        confidence = detect[i, 2]

        if confidence > 0.5:

            x1 = int(detect[i, 3]*w)
            y1 = int(detect[i, 4]*h)
            x2 = int(detect[i, 5]*w)
            y2 = int(detect[i, 6]*h)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            text = 'Face: {}%'.format(round(confidence*100, 2))
            cv2.putText(frame, text, (x1, y1-1), cv2.FONT_HERSHEY_COMPLEX,
                       1, (0, 0, 255), 1, cv2.LINE_AA)
    
    if not ret:
        print('frame open failed')
        break
    
#     edge = cv2.Canny(frame, 50, 150)
    
#     out.write(frame)
    
    cv2.imshow('image', frame)
#     cv2.imshow('edge', edge)
    
    if cv2.waitKey(20) == 27:
        break



# cv2.imshow('image', img)
# print(type(out))
# print(out.shape)

cap.release()
# cv2.waitKey()
cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
